In [129]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import os

# --- **ETAPA1: Integrando os dados (Merge)** ---

## 1. Carregamento dos DataFrames tratados


In [130]:
df_foc_queimadas_ro = pd.read_csv('../data/processados/tratados/df_foc_queimadas_ro_2019_2024_20250413.csv')
df_dados_met_ro = pd.read_csv('../data/processados/tratados/df_dados_met_ro_2020_2024_20250413.csv')
df_unid_local_bomb = pd.read_csv('../data/processados/tratados/focos_tratados-df_unid_local_bomb_20250406.csv')

## Função e aplicar nos três dataframes: Padroniza os dados para mitigar problemas de formatação no marge na chave principal que é **município**


In [131]:
import unicodedata

def padroniza_municipio(nome):
    if pd.isnull(nome):
        return nome
    nome = str(nome).strip().lower()  # remove espaços e põe tudo minúsculo
    nome = unicodedata.normalize('NFKD', nome).encode('ASCII', 'ignore').decode('utf-8')  # remove acentos
    nome = nome.title()  # converte para formato: Porto Velho, Cacoal, etc.
    return nome


In [132]:
df_foc_queimadas_ro['municipio'] = df_foc_queimadas_ro['municipio'].apply(padroniza_municipio)
df_dados_met_ro['municipio'] = df_dados_met_ro['municipio'].apply(padroniza_municipio)
df_unid_local_bomb['municipio'] = df_unid_local_bomb['cidade'].apply(padroniza_municipio)


In [133]:
df_foc_queimadas_ro.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma
0,1408346307,af46e7c0-6898-31a4-b688-0dc83f924e97,-9.597,-63.155,2019-11-06 17:40:00,Brasil,RONDÔNIA,Alto Paraiso,Amazônia
1,1408346306,abfcc502-192f-359b-ac7c-c877938c9983,-9.609,-63.245,2019-11-06 17:40:00,Brasil,RONDÔNIA,Alto Paraiso,Amazônia
2,1408346305,86269d85-0b7f-373a-8920-1813cdeaec46,-9.611,-63.409,2019-11-06 17:40:00,Brasil,RONDÔNIA,Alto Paraiso,Amazônia
3,1408346304,7143fbb1-e4b9-3e36-a155-95e40c2eee78,-9.634,-62.829,2019-11-06 17:40:00,Brasil,RONDÔNIA,Rio Crespo,Amazônia
4,1408346303,648bd7d5-5f78-376d-be5f-a55b60d25e64,-9.658,-63.301,2019-11-06 17:40:00,Brasil,RONDÔNIA,Alto Paraiso,Amazônia


In [134]:
df_dados_met_ro.head()

,data,hora_(utc),temp._ins._(c),temp._max._(c),temp._min._(c),umi._ins._(%),umi._max._(%),umi._min._(%),pto_orvalho_ins._(c),pto_orvalho_max._(c),pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio
0,2023-05-24,2100,29.8,30.9,29.8,67.0,67.0,61.0,22.9,23.4,22.2,997.8,997.8,997.5,0.6,158.0,2.5,713.8,0.0,Porto Velho
1,2023-05-24,2200,27.4,29.8,27.4,80.0,80.0,66.0,23.7,23.8,22.5,998.4,998.4,997.8,0.8,125.0,1.8,57.2,0.0,Porto Velho
2,2023-05-25,1100,24.1,24.1,23.4,95.0,95.0,94.0,23.1,23.1,22.4,1000.5,1000.5,1000.3,0.5,139.0,1.3,102.0,0.0,Porto Velho
3,2023-05-25,1200,25.6,25.6,24.1,92.0,95.0,92.0,24.3,24.3,23.2,1001.0,1001.0,1000.5,0.5,119.0,1.8,547.0,0.0,Porto Velho
4,2023-05-25,1300,27.5,27.6,25.6,79.0,92.0,79.0,23.5,24.2,23.2,1001.3,1001.3,1001.0,1.5,80.0,3.4,1348.0,0.0,Porto Velho


## Normalizar os nomes dos municípios

In [135]:
import unidecode

def padronizar_municipio(nome):
    if pd.isna(nome):
        return ''
    return unidecode.unidecode(nome.strip().upper().replace("'", "").replace("-", ""))


### Aplicando aos dataframes

In [136]:
df_foc_queimadas_ro['municipio_pad'] = df_foc_queimadas_ro['municipio'].apply(padronizar_municipio)
df_dados_met_ro['municipio_pad'] = df_dados_met_ro['municipio'].apply(padronizar_municipio)


In [137]:
focos_municipios = set(df_foc_queimadas_ro['municipio_pad'].unique())
met_municipios = set(df_dados_met_ro['municipio_pad'].unique())

municipios_nao_batem = focos_municipios - met_municipios
print("Municípios nos focos sem dados meteorológicos:", municipios_nao_batem)


Municípios nos focos sem dados meteorológicos: {'SAO FRANCISCO DO GUAPORE', 'CANDEIAS DO JAMARI', 'OURO PRETO DO OESTE', 'RIO CRESPO', 'THEOBROMA', 'ROLIM DE MOURA', 'COLORADO DO OESTE', 'MINISTRO ANDREAZZA', 'CEREJEIRAS', 'ALTO PARAISO', 'CUJUBIM', 'PARECIS', 'URUPA', 'COSTA MARQUES', 'MACHADINHO DOESTE', 'ITAPUA DO OESTE', 'NOVA BRASILANDIA DOESTE', 'PRESIDENTE MEDICI', 'SAO MIGUEL DO GUAPORE', 'CAMPO NOVO DE RONDONIA', 'PIMENTA BUENO', 'ALVORADA DOESTE', 'MIRANTE DA SERRA', 'GUAJARAMIRIM', 'PIMENTEIRAS DO OESTE', 'TEIXEIROPOLIS', 'JARU', 'CABIXI', 'BURITIS', 'PRIMAVERA DE RONDONIA', 'SANTA LUZIA DOESTE', 'SAO FELIPE DOESTE', 'GOVERNADOR JORGE TEIXEIRA', 'MONTE NEGRO', 'CORUMBIARA', 'SERINGUEIRAS', 'VALE DO PARAISO', 'NOVA MAMORE', 'NOVA UNIAO', 'CACAULANDIA', 'JIPARANA', 'CHUPINGUAIA', 'ESPIGAO DOESTE', 'ALTO ALEGRE DOS PARECIS', 'CASTANHEIRAS', 'NOVO HORIZONTE DO OESTE', 'ALTA FLORESTA DOESTE', 'VALE DO ANARI'}


### Municípios sem dados meteorológicos

In [138]:
pd.Series(list(municipios_nao_batem)).to_csv("../data/processados/municipios_sem_dados_meteorologicos.csv", index=False)


## 2. Renomeação de colunas meteorológicas


In [139]:
df_dados_met_ro.rename(columns={
    'Temp. Ins. (C)': 'temp_inst',
    'Temp. Max. (C)': 'temp_max',
    'Temp. Min. (C)': 'temp_min',
    'Umi. Ins. (%)': 'umid_inst',
    'Umi. Max. (%)': 'umid_max',
    'Umi. Min. (%)': 'umid_min',
    'Orvalho Ins. (C)': 'orvalho',
    'Ptc Máx. (C)': 'ptc_max',
    'Pressão Ins. (hPa)': 'pressao_inst',
    'Pressão Máx. (hPa)': 'pressao_max',
    'Pressão Mín. (hPa)': 'pressao_min',
    'Vel. Vento (m/s)': 'vento_vel',
    'Dir. Vento (graus)': 'vento_dir',
    'Raj. Vento (m/s)': 'vento_raj',
    'Radiação (KJ/m²)': 'radiacao',
    'Chuva (mm)': 'chuva',
    'Data (UTC)': 'data',
    'município': 'municipio'
}, inplace=True)


In [140]:
df_dados_met_ro.head()

,data,hora_(utc),temp._ins._(c),temp._max._(c),temp._min._(c),umi._ins._(%),umi._max._(%),umi._min._(%),pto_orvalho_ins._(c),pto_orvalho_max._(c),...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio,municipio_pad
0,2023-05-24,2100,29.8,30.9,29.8,67.0,67.0,61.0,22.9,23.4,...,997.8,997.8,997.5,0.6,158.0,2.5,713.8,0.0,Porto Velho,PORTO VELHO
1,2023-05-24,2200,27.4,29.8,27.4,80.0,80.0,66.0,23.7,23.8,...,998.4,998.4,997.8,0.8,125.0,1.8,57.2,0.0,Porto Velho,PORTO VELHO
2,2023-05-25,1100,24.1,24.1,23.4,95.0,95.0,94.0,23.1,23.1,...,1000.5,1000.5,1000.3,0.5,139.0,1.3,102.0,0.0,Porto Velho,PORTO VELHO
3,2023-05-25,1200,25.6,25.6,24.1,92.0,95.0,92.0,24.3,24.3,...,1001.0,1001.0,1000.5,0.5,119.0,1.8,547.0,0.0,Porto Velho,PORTO VELHO
4,2023-05-25,1300,27.5,27.6,25.6,79.0,92.0,79.0,23.5,24.2,...,1001.3,1001.3,1001.0,1.5,80.0,3.4,1348.0,0.0,Porto Velho,PORTO VELHO


In [141]:
print(df_dados_met_ro.shape[0])

28419


## 📆 Tratamento das datas

In [142]:
df_dados_met_ro['data'] = pd.to_datetime(df_dados_met_ro['data'], errors='coerce')
df_foc_queimadas_ro['data'] = pd.to_datetime(df_foc_queimadas_ro['data'], errors='coerce')

### Padronizando as datas para o merge formato YYYY-MM-DD

In [143]:
df_foc_queimadas_ro['data'] = pd.to_datetime(df_foc_queimadas_ro['data']).dt.date
df_dados_met_ro['data'] = pd.to_datetime(df_dados_met_ro['data']).dt.date


### Merge entre Focos de Queimada e Dados Meteorológicos

___

## Nova coluna data_formatada para compor o Marge

In [144]:
df_foc_queimadas_ro['data_formatada'] = pd.to_datetime(df_foc_queimadas_ro['data']).dt.date
df_dados_met_ro['data_formatada'] = pd.to_datetime(df_dados_met_ro['data']).dt.date


In [145]:
# Até que enfim cheguei aqui...😩😩🤯
# Merge pela alma do projeto: data + município
df_merge = pd.merge(
    df_foc_queimadas_ro,
    df_dados_met_ro,
    left_on=['data_formatada', 'municipio'],
    right_on=['data_formatada', 'municipio'],
    how='left'
)

df_merge.head()

,id_bdq,foco_id,lat,lon,data_x,pais,estado,municipio,bioma,municipio_pad_x,...,pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y
0,1408346307,af46e7c0-6898-31a4-b688-0dc83f924e97,-9.597,-63.155,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1408346306,abfcc502-192f-359b-ac7c-c877938c9983,-9.609,-63.245,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1408346305,86269d85-0b7f-373a-8920-1813cdeaec46,-9.611,-63.409,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1408346304,7143fbb1-e4b9-3e36-a155-95e40c2eee78,-9.634,-62.829,2019-11-06,Brasil,RONDÔNIA,Rio Crespo,Amazônia,RIO CRESPO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1408346303,648bd7d5-5f78-376d-be5f-a55b60d25e64,-9.658,-63.301,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
df_completos = df_merge.dropna()
df_completos.head()

,id_bdq,foco_id,lat,lon,data_x,pais,estado,municipio,bioma,municipio_pad_x,...,pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y
11270,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.1,992.3,992.3,991.6,0.4,277.0,0.9,91.9,0.0,CACOAL
11271,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.4,992.8,992.8,992.3,0.4,271.0,1.2,745.0,0.0,CACOAL
11272,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,19.1,993.2,993.2,992.8,1.4,56.0,4.0,1452.0,0.0,CACOAL
11273,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,17.6,993.0,993.2,992.9,2.7,40.0,6.5,2169.8,0.0,CACOAL
11274,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,16.2,992.7,993.0,992.7,3.5,36.0,7.2,2580.6,0.0,CACOAL


## Lista apenas colunas com dados meteorológicos

In [147]:
# Lista das colunas meteorológicas
colunas_meteorologicas = [
    'hora_(utc)', 'temp._ins._(c)', 'temp._max._(c)', 'temp._min._(c)',
    'umi._ins._(%)', 'umi._max._(%)', 'umi._min._(%)',
    'pto_orvalho_ins._(c)', 'pto_orvalho_max._(c)', 'pto_orvalho_min._(c)',
    'pressao_ins._(hpa)', 'pressao_max._(hpa)', 'pressao_min._(hpa)',
    'vel._vento_(m/s)', 'dir._vento_(m/s)', 'raj._vento_(m/s)',
    'radiacao_(kj/m²)', 'chuva_(mm)'
]

df_base_modelo_incendios  = df_merge.dropna(subset=colunas_meteorologicas)


In [148]:
df_base_modelo_incendios .head()

,id_bdq,foco_id,lat,lon,data_x,pais,estado,municipio,bioma,municipio_pad_x,...,pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y
11270,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.1,992.3,992.3,991.6,0.4,277.0,0.9,91.9,0.0,CACOAL
11271,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.4,992.8,992.8,992.3,0.4,271.0,1.2,745.0,0.0,CACOAL
11272,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,19.1,993.2,993.2,992.8,1.4,56.0,4.0,1452.0,0.0,CACOAL
11273,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,17.6,993.0,993.2,992.9,2.7,40.0,6.5,2169.8,0.0,CACOAL
11274,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,16.2,992.7,993.0,992.7,3.5,36.0,7.2,2580.6,0.0,CACOAL


In [149]:
print(df_base_modelo_incendios .shape[0])

61997


## ----  Validação Pós-Merge ----

### 1. Verificar dados ausentes (NaNs)

In [150]:
# Verificar quantidade de valores ausentes por coluna
df_base_modelo_incendios.isna().sum().sort_values(ascending=False)


id_bdq                  0
umi._ins._(%)           0
chuva_(mm)              0
radiacao_(kj/m²)        0
raj._vento_(m/s)        0
dir._vento_(m/s)        0
vel._vento_(m/s)        0
pressao_min._(hpa)      0
pressao_max._(hpa)      0
pressao_ins._(hpa)      0
pto_orvalho_min._(c)    0
pto_orvalho_max._(c)    0
pto_orvalho_ins._(c)    0
umi._min._(%)           0
umi._max._(%)           0
temp._min._(c)          0
foco_id                 0
temp._max._(c)          0
temp._ins._(c)          0
hora_(utc)              0
data_y                  0
data_formatada          0
municipio_pad_x         0
bioma                   0
municipio               0
estado                  0
pais                    0
data_x                  0
lon                     0
lat                     0
municipio_pad_y         0
dtype: int64

### 2. Verificar se existem linhas duplicadas

In [151]:
# Verifica se há registros duplicados completos
duplicados = df_base_modelo_incendios.duplicated()
print(f"Total de linhas duplicadas: {duplicados.sum()}")


Total de linhas duplicadas: 0


## Renomeei a data de novo aqui 😅

In [152]:
# Renomear a coluna data_x para data
df_base_modelo_incendios.rename(columns={"data_x": "data"}, inplace=True)


C:\Users\ander\AppData\Local\Temp\ipykernel_5912\3629414703.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base_modelo_incendios.rename(columns={"data_x": "data"}, inplace=True)


In [153]:
df_base_modelo_incendios.head()  # Exibir as primeiras 10 linhas do DataFrame

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pto_orvalho_min._(c),pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y
11270,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.1,992.3,992.3,991.6,0.4,277.0,0.9,91.9,0.0,CACOAL
11271,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,18.4,992.8,992.8,992.3,0.4,271.0,1.2,745.0,0.0,CACOAL
11272,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,19.1,993.2,993.2,992.8,1.4,56.0,4.0,1452.0,0.0,CACOAL
11273,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,17.6,993.0,993.2,992.9,2.7,40.0,6.5,2169.8,0.0,CACOAL
11274,1500144484,0a71cec0-6ac5-31eb-9b3d-c9ad4ce16668,-11.538,-61.612,2020-06-21,Brasil,RONDÔNIA,Cacoal,Amazônia,CACOAL,...,16.2,992.7,993.0,992.7,3.5,36.0,7.2,2580.6,0.0,CACOAL


### 3. Verificar formato de colunas chave

In [154]:
df_base_modelo_incendios['data'] = pd.to_datetime(df_base_modelo_incendios['data'], errors='coerce')
print(df_base_modelo_incendios['data'].dtypes)


datetime64[ns]


C:\Users\ander\AppData\Local\Temp\ipykernel_5912\817531445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base_modelo_incendios['data'] = pd.to_datetime(df_base_modelo_incendios['data'], errors='coerce')


### 4. Verificar se há dados de múltiplos anos, municípios e datas esperadas

In [155]:
# Quantidade de municípios
print("Municípios únicos:", df_base_modelo_incendios['municipio'].nunique())

# Amostragem de anos
print("Anos na base:", df_base_modelo_incendios['data'].dt.year.unique())

# Checar se há dados para todos os anos esperados (ex: 2019 a 2024)
anos_esperados = set(range(2019, 2025))
anos_presentes = set(df_base_modelo_incendios['data'].dt.year.unique())
print("Anos ausentes:", anos_esperados - anos_presentes)


Municípios únicos: 4
Anos na base: [2020 2023 2024]
Anos ausentes: {2019, 2021, 2022}


# ---- DEPOIS DAQUI, PERCEBI QUE ESSE MERGE NÃO IRIA ROLAR, POIS OS DADOS DE FOCOS NÃO ESTÃO SE CONCILIANDO CORRETAMENTE COM OS DADOS METEOROLÓGICOS

# --- **MUDANÇA DE PLANOS COM O (Merge) A PARTIR DAQUI** ---

### 🧩 PLANO DE AÇÃO: Agrupar dados meteorológicos por município de referência

Vamos assumir que alguns municípios vizinhos podem usar os dados meteorológicos da estação mais próxima.

## Mapeamento da proximidade geográfica e facilidade logística

## 1. Criar uma coluna "estação de referência" para os municípios sem dados meteorológicos

In [156]:
mapeamento_estacoes = {
    # Região Central (Proximidade: Cacoal)
    'Ji-Parana': 'Cacoal',
    'Pimenta Bueno': 'Cacoal',
    'Rolim De Moura': 'Cacoal',
    'Alta Floresta D\'Oeste': 'Cacoal',
    'Santa Luzia D\'Oeste': 'Cacoal',
    'Espigao D\'Oeste': 'Cacoal',
    'Ministro Andreazza': 'Cacoal',
    'Parecis': 'Cacoal',
    'Primavera De Rondonia': 'Cacoal',
    'Novo Horizonte Do Oeste': 'Cacoal',
    'Castanheiras': 'Cacoal',
    'Sao Felipe D\'Oeste': 'Cacoal',
    
    # Região Norte (Proximidade: Ariquemes)
    'Ariquemes': 'Ariquemes',  # já é base
    'Jaru': 'Ariquemes',
    'Cujubim': 'Ariquemes',
    'Buritis': 'Ariquemes',
    'Monte Negro': 'Ariquemes',
    'Machadinho D\'Oeste': 'Ariquemes',
    'Campo Novo De Rondonia': 'Ariquemes',
    'Cacaulandia': 'Ariquemes',
    'Theobroma': 'Ariquemes',
    'Vale Do Anari': 'Ariquemes',
    'Vale Do Paraiso': 'Ariquemes',
    'Rio Crespo': 'Ariquemes',
    'Nova Uniao': 'Ariquemes',
    'Governador Jorge Teixeira': 'Ariquemes',
    
    # Região Metropolitana (Porto Velho)
    'Porto Velho': 'Porto Velho',  # já é base
    'Guajara-Mirim': 'Porto Velho',
    'Candeias Do Jamari': 'Porto Velho',
    'Itapua Do Oeste': 'Porto Velho',
    
    # Extremo Oeste (Proximidade: Porto Velho)
    'Costa Marques': 'Porto Velho',
    'Sao Francisco Do Guapore': 'Porto Velho',
    'Sao Miguel Do Guapore': 'Porto Velho',
    'Seringueiras': 'Porto Velho',
    'Alvorada D\'Oeste': 'Porto Velho',
    
    # Extremo Sul (Proximidade: Vilhena)
    'Vilhena': 'Vilhena',  # já é base
    'Colorado Do Oeste': 'Vilhena',
    'Cerejeiras': 'Vilhena',
    'Cabixi': 'Vilhena',
    'Pimenteiras Do Oeste': 'Vilhena',
    'Corumbiara': 'Vilhena',
    'Chupinguaia': 'Vilhena',
    'Alto Alegre Dos Parecis': 'Vilhena',
    
    # Região Centro-Norte/Sul (ajuste por contexto)
    'Presidente Medici': 'Cacoal',
    'Ouro Preto Do Oeste': 'Cacoal',
    'Nova Brasilandia D\'Oeste': 'Cacoal',
    'Urupa': 'Cacoal',
    'Mirante Da Serra': 'Cacoal',
    'Alto Paraiso': 'Ariquemes',
}


## 2. Nova coluna no dataset de focos

### Preparar df_dados_met_ro para o merge

In [157]:

df_dados_met_ro.rename(columns={'municipio': 'estacao_ref', 'Data (UTC)': 'data'}, inplace=True)

# Converter datas
df_foc_queimadas_ro['data'] = pd.to_datetime(df_foc_queimadas_ro['data'])
df_dados_met_ro['data'] = pd.to_datetime(df_dados_met_ro['data'])


Esse código abaixo é muito show! Preciso usar ele para uma outra ideia

In [158]:
df_foc_queimadas_ro['estacao_ref'] = df_foc_queimadas_ro['municipio'].map(mapeamento_estacoes)


In [159]:
df_foc_queimadas_ro.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad,data_formatada,estacao_ref
0,1408346307,af46e7c0-6898-31a4-b688-0dc83f924e97,-9.597,-63.155,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,2019-11-06,Ariquemes
1,1408346306,abfcc502-192f-359b-ac7c-c877938c9983,-9.609,-63.245,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,2019-11-06,Ariquemes
2,1408346305,86269d85-0b7f-373a-8920-1813cdeaec46,-9.611,-63.409,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,2019-11-06,Ariquemes
3,1408346304,7143fbb1-e4b9-3e36-a155-95e40c2eee78,-9.634,-62.829,2019-11-06,Brasil,RONDÔNIA,Rio Crespo,Amazônia,RIO CRESPO,2019-11-06,Ariquemes
4,1408346303,648bd7d5-5f78-376d-be5f-a55b60d25e64,-9.658,-63.301,2019-11-06,Brasil,RONDÔNIA,Alto Paraiso,Amazônia,ALTO PARAISO,2019-11-06,Ariquemes


## 3. Fazer o merge com base em estacao_ref + data (Agora sim 😎✅🎲)

In [160]:
df_foc_queimadas_ro['data'] = pd.to_datetime(df_foc_queimadas_ro['data'])
df_dados_met_ro['data'] = pd.to_datetime(df_dados_met_ro['data'])

df_dados_met_ro.rename(columns={'municipio': 'estacao_ref'}, inplace=True)

df_base_modelo_incendios = pd.merge(
    df_foc_queimadas_ro,
    df_dados_met_ro,
    on=['estacao_ref', 'data'],
    how='inner'  # ou 'left' se quiser manter todos os focos e só pegar meteorologia onde houver
)



### Replicação do Mapeamento

In [161]:
df_foc_queimadas_ro['estacao_ref'] = df_foc_queimadas_ro['municipio'].map(mapeamento_estacoes)


### Atualizar o mapeamento_estacoes para os que faltam

In [162]:
# Atualizando o mapeamento com os casos restantes
mapeamento_estacoes.update({
    'Nova Mamore': 'Porto Velho',     # Município vizinho, proximidade geográfica com estação PVH
    'Cacoal': 'Cacoal',               # Aqui é fácil, já existe a estação em Cacoal!
    'Teixeiropolis': 'Ariquemes',     # Fica ali na região central-norte, mais próximo de Ariquemes
})


In [163]:
df_foc_queimadas_ro['estacao_ref'] = df_foc_queimadas_ro['municipio'].map(mapeamento_estacoes)

### checar os que ficaram de fora

## Esse código abaixo é que determina a vitória do merge. Se realmente o merge deu certo!!

In [164]:
faltantes = df_foc_queimadas_ro[df_foc_queimadas_ro['estacao_ref'].isna()]
print(f"Focos SEM estação meteorológica vinculada: {faltantes['municipio'].unique()}")


Focos SEM estação meteorológica vinculada: []


In [165]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y,data_formatada_y
0,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.1,992.1,991.6,1.2,152.0,3.6,96.9,0.0,CACOAL,2020-06-19
1,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,992.8,992.1,1.4,139.0,2.6,713.0,0.0,CACOAL,2020-06-19
2,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,993.0,993.0,992.8,1.5,132.0,3.0,1446.2,0.0,CACOAL,2020-06-19
3,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,993.0,992.8,2.0,114.0,3.9,2146.4,0.0,CACOAL,2020-06-19
4,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.6,992.8,992.6,4.3,119.0,7.7,2575.2,0.0,CACOAL,2020-06-19


In [166]:
print(df_base_modelo_incendios.shape[0])  # Total de registros após o merge

150008


In [167]:
df_base_modelo_incendios.iloc[38234:38238]

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),municipio_pad_y,data_formatada_y
38234,1657894133,e0453c10-4b80-367e-9f8d-f2fd744b7486,-13.07176,-61.63435,2023-10-24,Brasil,RONDÔNIA,Cerejeiras,Amazônia,CEREJEIRAS,...,944.7,945.9,944.7,2.0,304.0,5.6,3258.2,0.0,VILHENA,2023-10-24
38235,1657894133,e0453c10-4b80-367e-9f8d-f2fd744b7486,-13.07176,-61.63435,2023-10-24,Brasil,RONDÔNIA,Cerejeiras,Amazônia,CEREJEIRAS,...,943.4,944.6,943.4,2.0,339.0,5.5,2933.1,0.0,VILHENA,2023-10-24
38236,1657894133,e0453c10-4b80-367e-9f8d-f2fd744b7486,-13.07176,-61.63435,2023-10-24,Brasil,RONDÔNIA,Cerejeiras,Amazônia,CEREJEIRAS,...,942.8,943.4,942.8,2.3,330.0,5.1,1866.0,0.0,VILHENA,2023-10-24
38237,1657894133,e0453c10-4b80-367e-9f8d-f2fd744b7486,-13.07176,-61.63435,2023-10-24,Brasil,RONDÔNIA,Cerejeiras,Amazônia,CEREJEIRAS,...,941.8,942.8,941.8,1.7,328.0,5.1,1384.4,0.0,VILHENA,2023-10-24


## 🔧 Etapa de limpeza pós-merge 

In [168]:

df_base_modelo_incendios = df_base_modelo_incendios.rename(columns={
    'municipio_pad_y': 'estacao_ref'
})


### Remover a coluna data do merge anterior

In [169]:
# Renomear a coluna data 
df_base_modelo_incendios = df_base_modelo_incendios.rename(columns={'data_formatada': 'data'})

In [170]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),estacao_ref,data_formatada_y
0,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.1,992.1,991.6,1.2,152.0,3.6,96.9,0.0,CACOAL,2020-06-19
1,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,992.8,992.1,1.4,139.0,2.6,713.0,0.0,CACOAL,2020-06-19
2,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,993.0,993.0,992.8,1.5,132.0,3.0,1446.2,0.0,CACOAL,2020-06-19
3,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,993.0,992.8,2.0,114.0,3.9,2146.4,0.0,CACOAL,2020-06-19
4,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.6,992.8,992.6,4.3,119.0,7.7,2575.2,0.0,CACOAL,2020-06-19


## --- 🧪 **Verifca se os dados meteorológicos estão consistentes por data e estação: ** ---

In [171]:
df_base_modelo_incendios[['data', 'municipio', 'estacao_ref', 'chuva_(mm)', 'temp._ins._(c)', 'umi._ins._(%)']]


,data,municipio,estacao_ref,estacao_ref,chuva_(mm),temp._ins._(c),umi._ins._(%)
0,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,20.0,90.0
1,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,22.7,82.0
2,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,26.1,69.0
3,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,30.1,54.0
4,2020-06-19,Pimenta Bueno,Cacoal,CACOAL,0.0,31.4,45.0
...,...,...,...,...,...,...,...
150003,2024-12-30,Itapua Do Oeste,Porto Velho,PORTO VELHO,0.0,29.6,69.0
150004,2024-12-30,Itapua Do Oeste,Porto Velho,PORTO VELHO,0.0,31.5,61.0
150005,2024-12-30,Itapua Do Oeste,Porto Velho,PORTO VELHO,0.0,31.5,59.0
150006,2024-12-30,Itapua Do Oeste,Porto Velho,PORTO VELHO,0.0,32.1,58.0


## Aqui eu trago apenas os dados com chuvaa maior que 0, ou seja, com chuva. 🌧

In [172]:
# Filtrar os dados onde a coluna 'chuva_(mm)' é maior que 0
dados_com_chuva = df_base_modelo_incendios[df_base_modelo_incendios['chuva_(mm)'] > 0]

# Exibir os primeiros registros do resultado
dados_com_chuva.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),estacao_ref,data_formatada_y
32,1521156494,158e8837-a683-3d2c-9c94-888072ef186f,-11.230,-60.828,2020-12-03,Brasil,RONDÔNIA,Espigao D'Oeste,Amazônia,ESPIGAO DOESTE,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03
58,1521156574,e1cf038a-3f45-3faf-a299-3a1b93744060,-11.136,-61.558,2020-12-03,Brasil,RONDÔNIA,Ministro Andreazza,Amazônia,MINISTRO ANDREAZZA,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03
71,1521156532,fbe55e32-a0dd-30f1-8236-ce2288cd1dae,-11.284,-60.992,2020-12-03,Brasil,RONDÔNIA,Espigao D'Oeste,Amazônia,ESPIGAO DOESTE,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03
84,1521156471,37ad6f07-c6a2-3a29-aa30-c860484cf88a,-11.879,-61.179,2020-12-03,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03
97,1521156745,9877826f-c459-3546-b58b-6061842a0d72,-11.486,-62.130,2020-12-03,Brasil,RONDÔNIA,Nova Brasilandia D'Oeste,Amazônia,NOVA BRASILANDIA DOESTE,...,986.7,988.3,986.7,2.1,29.0,14.2,1464.1,4.8,CACOAL,2020-12-03


In [173]:
print(dados_com_chuva.shape[0])

2388


# ----**ETAPA 2: IDENTIFICANDO VARIÁVEIS**----

In [174]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_ins._(hpa),pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),chuva_(mm),estacao_ref,data_formatada_y
0,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.1,992.1,991.6,1.2,152.0,3.6,96.9,0.0,CACOAL,2020-06-19
1,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,992.8,992.1,1.4,139.0,2.6,713.0,0.0,CACOAL,2020-06-19
2,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,993.0,993.0,992.8,1.5,132.0,3.0,1446.2,0.0,CACOAL,2020-06-19
3,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,993.0,992.8,2.0,114.0,3.9,2146.4,0.0,CACOAL,2020-06-19
4,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.6,992.8,992.6,4.3,119.0,7.7,2575.2,0.0,CACOAL,2020-06-19


### Removendo colunas duplicadas no dataframe

In [175]:
# Remove colunas duplicadas do DataFrame
df_base_modelo_incendios = df_base_modelo_incendios.loc[:, ~df_base_modelo_incendios.columns.duplicated()]

# Exibe as colunas restantes para verificar
print(df_base_modelo_incendios.columns)

Index(['id_bdq', 'foco_id', 'lat', 'lon', 'data', 'pais', 'estado',
       'municipio', 'bioma', 'municipio_pad_x', 'data_formatada_x',
       'estacao_ref', 'hora_(utc)', 'temp._ins._(c)', 'temp._max._(c)',
       'temp._min._(c)', 'umi._ins._(%)', 'umi._max._(%)', 'umi._min._(%)',
       'pto_orvalho_ins._(c)', 'pto_orvalho_max._(c)', 'pto_orvalho_min._(c)',
       'pressao_ins._(hpa)', 'pressao_max._(hpa)', 'pressao_min._(hpa)',
       'vel._vento_(m/s)', 'dir._vento_(m/s)', 'raj._vento_(m/s)',
       'radiacao_(kj/m²)', 'chuva_(mm)', 'data_formatada_y'],
      dtype='object')


## 🧪 Feature Engineering 🔬

### **🧩 1. temp_media**

Adiciona ao Dataset a Média da temperatura instantânea, máxima e mínima

In [176]:
df_base_modelo_incendios['temp_media'] = df_base_modelo_incendios[[
    'temp._ins._(c)', 'temp._max._(c)', 'temp._min._(c)'
]].mean(axis=1)


### **🧩 2. umidade_relativa**

Adiciona a umidade_relativa média entre instantânea, máxima e mínima

In [177]:
df_base_modelo_incendios['umidade_relativa'] = df_base_modelo_incendios[[
    'umi._ins._(%)', 'umi._max._(%)', 'umi._min._(%)'
]].mean(axis=1)


### **🧩 3. precipitacao**

Só renomeia a coluna "chuva_(mm)" para precipitcao

In [178]:
df_base_modelo_incendios.rename(columns={'chuva_(mm)':'precipitacao'}, inplace=True)

In [179]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,pressao_max._(hpa),pressao_min._(hpa),vel._vento_(m/s),dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),precipitacao,data_formatada_y,temp_media,umidade_relativa
0,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.1,991.6,1.2,152.0,3.6,96.9,0.0,2020-06-19,19.733333,90.666667
1,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,992.1,1.4,139.0,2.6,713.0,0.0,2020-06-19,21.800000,84.666667
2,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,993.0,992.8,1.5,132.0,3.0,1446.2,0.0,2020-06-19,25.000000,73.666667
3,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,993.0,992.8,2.0,114.0,3.9,2146.4,0.0,2020-06-19,28.800000,59.000000
4,1500003545,ac8abed5-6e41-3592-abc8-d29eff3faf96,-11.502,-60.486,2020-06-19,Brasil,RONDÔNIA,Pimenta Bueno,Amazônia,PIMENTA BUENO,...,992.8,992.6,4.3,119.0,7.7,2575.2,0.0,2020-06-19,31.166667,47.666667


### **🧩 4. dias_sem_chuva**

Agrupa por município e ordenar por data, acumulando dias consecutivos com zero de chuva. Essa parte aqui é muito importante

In [250]:
def contar_dias_sem_chuva(grupo):
    cont = 0
    dias_sem_chuva = []
    for choveu in grupo:
        if choveu:
            cont = 0
        else:
            cont += 1
        dias_sem_chuva.append(cont)
    return dias_sem_chuva


In [259]:
# Passo 1: agregando por município e data (usando max para garantir que, se tiver qualquer chuva, conta)
df_diario = (
    df_base_modelo_incendios
    .groupby(['municipio_pad_x', 'data'])
    .agg({'precipitacao': 'max'})  # ou 'sum', depende da sua definição de "teve chuva"
    .reset_index()
)

# Passo 2: criando a coluna "choveu"
df_diario['choveu'] = df_diario['precipitacao'] > 0

# Passo 3: calcular dias sem chuva por município corretamente
df_diario['dias_sem_chuva'] = (
    df_diario
    .groupby('municipio_pad_x')['choveu']
    .transform(contar_dias_sem_chuva)
)

# Agora você pode mesclar de volta com o DataFrame original, se quiser
df_base_corrigido = pd.merge(
    df_base_modelo_incendios,
    df_diario[['municipio_pad_x', 'data', 'dias_sem_chuva']],
    on=['municipio_pad_x', 'data'],
    how='left'
)


In [261]:
# Merge limpo, trazendo apenas o 'dias_sem_chuva' calculado corretamente
df_base_corrigido = pd.merge(
    df_base_modelo_incendios.drop(columns=['dias_sem_chuva'], errors='ignore'),
    df_diario[['municipio_pad_x', 'data', 'dias_sem_chuva']],
    on=['municipio_pad_x', 'data'],
    how='left'
)


In [264]:
df_base_corrigido.columns


Index(['id_bdq', 'foco_id', 'lat', 'lon', 'data', 'pais', 'estado',
       'municipio', 'bioma', 'municipio_pad_x', 'data_formatada_x',
       'estacao_ref', 'hora_(utc)', 'temp._ins._(c)', 'temp._max._(c)',
       'temp._min._(c)', 'umi._ins._(%)', 'umi._max._(%)', 'umi._min._(%)',
       'pto_orvalho_ins._(c)', 'pto_orvalho_max._(c)', 'pto_orvalho_min._(c)',
       'pressao_ins._(hpa)', 'pressao_max._(hpa)', 'pressao_min._(hpa)',
       'vel._vento_(m/s)', 'dir._vento_(m/s)', 'raj._vento_(m/s)',
       'radiacao_(kj/m²)', 'precipitacao', 'data_formatada_y', 'temp_media',
       'umidade_relativa', 'choveu', 'intensidade_chuva', 'dias_sem_chuva_x',
       'fumaca_media_x', 'risco_fogo', 'fumaca_media_y', 'dias_sem_chuva_y',
       'dias_sem_chuva'],
      dtype='object')

In [287]:
df_base_corrigido.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva_corrigido,dias_sem_chuva
0,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,10.4,17.9,7.6,21.966667,89.666667,True,Chuva moderada,11.886667,0,0
1,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.9,354.3,0.2,22.900000,90.666667,True,Chuva leve,11.960000,0,0
2,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,2.9,1031.8,0.0,25.133333,81.333333,False,Sem chuva,15.953333,0,0
3,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,4.0,1915.0,0.0,27.933333,68.000000,False,Sem chuva,21.373333,0,0
4,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.7,2263.9,0.0,28.833333,63.333333,False,Sem chuva,23.433333,0,0


In [294]:
df_base_corrigido.drop(columns=['dias_sem_chuva_corrigido'], inplace=True, errors='ignore')

In [285]:
df_base_corrigido['dias_sem_chuva'] = df_base_corrigido['dias_sem_chuva_y']

df_base_corrigido = df_base_corrigido.drop(columns=[
    'dias_sem_chuva_x', 
    'dias_sem_chuva_y', 
    'data_formatada_x', 
    'data_formatada_y', 
    'fumaca_media_x', 
    'fumaca_media_y'  # <- Se você não quiser duplicidade
])


In [297]:
df_base_corrigido[df_base_corrigido['dias_sem_chuva'] > 60]


,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva
108481,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,212.0,2.7,35.8,0.0,18.700000,80.333333,False,Sem chuva,4337.880000,61
108482,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,171.0,2.1,386.3,0.0,21.266667,67.666667,False,Sem chuva,4343.006667,61
108483,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,129.0,3.1,1042.5,0.0,25.366667,51.666667,False,Sem chuva,4349.746667,61
108484,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,148.0,5.6,1886.7,0.0,29.533333,34.333333,False,Sem chuva,4356.913333,61
108485,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,120.0,5.7,2474.9,0.0,32.366667,25.666667,False,Sem chuva,4360.946667,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110348,1710115195,a3d8b170-29a3-39c4-be3d-59e4b1afabfa,-9.65949,-64.96112,2024-08-15,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,314.0,4.9,2206.1,0.0,35.233333,29.000000,False,Sem chuva,4919.993333,62
110349,1710115195,a3d8b170-29a3-39c4-be3d-59e4b1afabfa,-9.65949,-64.96112,2024-08-15,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,344.0,4.7,1713.9,0.0,35.700000,28.666667,False,Sem chuva,4920.580000,62
110350,1710115195,a3d8b170-29a3-39c4-be3d-59e4b1afabfa,-9.65949,-64.96112,2024-08-15,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,9.0,6.3,1186.7,0.0,35.333333,29.333333,False,Sem chuva,4920.533333,62
110351,1710115195,a3d8b170-29a3-39c4-be3d-59e4b1afabfa,-9.65949,-64.96112,2024-08-15,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,338.0,3.8,505.3,0.0,34.766667,32.666667,False,Sem chuva,4919.606667,62


In [298]:
df_base_corrigido.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva
0,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,295.0,10.4,17.9,7.6,21.966667,89.666667,True,Chuva moderada,11.886667,0
1,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,313.0,3.9,354.3,0.2,22.900000,90.666667,True,Chuva leve,11.960000,0
2,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,149.0,2.9,1031.8,0.0,25.133333,81.333333,False,Sem chuva,15.953333,0
3,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,131.0,4.0,1915.0,0.0,27.933333,68.000000,False,Sem chuva,21.373333,0
4,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,171.0,3.7,2263.9,0.0,28.833333,63.333333,False,Sem chuva,23.433333,0


In [300]:
df_base_modelo_incendios = df_base_corrigido.copy()

In [301]:
df_base_modelo_incendios.iloc[38520:38630]

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva
38520,1652982859,4ba6c69d-53cc-37d2-8e49-025cf66fe4da,-12.42467,-63.88049,2023-09-25,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,280.0,3.1,673.3,0.0,28.233333,72.666667,False,Sem chuva,45.593333,0
38521,1652982859,4ba6c69d-53cc-37d2-8e49-025cf66fe4da,-12.42467,-63.88049,2023-09-25,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,80.0,5.8,1272.2,0.0,35.233333,42.333333,False,Sem chuva,57.793333,0
38522,1652982859,4ba6c69d-53cc-37d2-8e49-025cf66fe4da,-12.42467,-63.88049,2023-09-25,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,227.0,10.6,444.5,0.2,30.133333,63.000000,True,Chuva leve,23.153333,0
38523,1652982859,4ba6c69d-53cc-37d2-8e49-025cf66fe4da,-12.42467,-63.88049,2023-09-25,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,207.0,4.8,927.3,0.0,29.633333,69.000000,False,Sem chuva,21.453333,0
38524,1652982859,4ba6c69d-53cc-37d2-8e49-025cf66fe4da,-12.42467,-63.88049,2023-09-25,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,225.0,3.2,1118.6,0.0,31.833333,61.333333,False,Sem chuva,24.933333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38625,1653342897,82704609-a45d-31ff-927c-1a9bfb8e1adb,-12.42302,-63.87969,2023-09-27,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,242.0,1.3,708.0,0.0,28.966667,76.333333,False,Sem chuva,19.886667,0
38626,1653342897,82704609-a45d-31ff-927c-1a9bfb8e1adb,-12.42302,-63.87969,2023-09-27,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,264.0,6.3,2503.5,0.0,36.333333,38.333333,False,Sem chuva,34.533333,0
38627,1653342897,82704609-a45d-31ff-927c-1a9bfb8e1adb,-12.42302,-63.87969,2023-09-27,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,193.0,20.4,460.8,9.6,29.133333,57.333333,True,Chuva moderada,24.453333,0
38628,1653342897,82704609-a45d-31ff-927c-1a9bfb8e1adb,-12.42302,-63.87969,2023-09-27,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,163.0,2.2,975.9,0.0,34.933333,47.666667,False,Sem chuva,29.973333,0


In [310]:
df_base_modelo_incendios[df_base_modelo_incendios['dias_sem_chuva'] > 60]


,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva
108481,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,212.0,2.7,35.8,0.0,18.700000,80.333333,False,Sem chuva,4337.880000,61
108482,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,171.0,2.1,386.3,0.0,21.266667,67.666667,False,Sem chuva,4343.006667,61
108483,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,129.0,3.1,1042.5,0.0,25.366667,51.666667,False,Sem chuva,4349.746667,61
108484,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,148.0,5.6,1886.7,0.0,29.533333,34.333333,False,Sem chuva,4356.913333,61
108485,1709723175,b5979b94-bd46-3642-ba01-8c7298ac2fd4,-9.80752,-66.74870,2024-08-14,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,120.0,5.7,2474.9,0.0,32.366667,25.666667,False,Sem chuva,4360.946667,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110348,1710115195,a3d8b170-29a3-39c4-be3d-59e4b1afabfa,-9.65949,-64.96112,2024-08-15,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,314.0,4.9,2206.1,0.0,35.233333,29.000000,False,Sem chuva,4919.993333,62
110349,1710115195,a3d8b170-29a3-39c4-be3d-59e4b1afabfa,-9.65949,-64.96112,2024-08-15,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,344.0,4.7,1713.9,0.0,35.700000,28.666667,False,Sem chuva,4920.580000,62
110350,1710115195,a3d8b170-29a3-39c4-be3d-59e4b1afabfa,-9.65949,-64.96112,2024-08-15,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,9.0,6.3,1186.7,0.0,35.333333,29.333333,False,Sem chuva,4920.533333,62
110351,1710115195,a3d8b170-29a3-39c4-be3d-59e4b1afabfa,-9.65949,-64.96112,2024-08-15,Brasil,RONDÔNIA,Porto Velho,Amazônia,PORTO VELHO,...,338.0,3.8,505.3,0.0,34.766667,32.666667,False,Sem chuva,4919.606667,62


In [305]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva
0,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,295.0,10.4,17.9,7.6,21.966667,89.666667,True,Chuva moderada,11.886667,0
1,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,313.0,3.9,354.3,0.2,22.900000,90.666667,True,Chuva leve,11.960000,0
2,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,149.0,2.9,1031.8,0.0,25.133333,81.333333,False,Sem chuva,15.953333,0
3,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,131.0,4.0,1915.0,0.0,27.933333,68.000000,False,Sem chuva,21.373333,0
4,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,171.0,3.7,2263.9,0.0,28.833333,63.333333,False,Sem chuva,23.433333,0


In [311]:
df_base_modelo_incendios

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,dir._vento_(m/s),raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva
0,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38000,-62.2060,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,295.0,10.4,17.9,7.6,21.966667,89.666667,True,Chuva moderada,11.886667,0
1,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38000,-62.2060,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,313.0,3.9,354.3,0.2,22.900000,90.666667,True,Chuva leve,11.960000,0
2,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38000,-62.2060,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,149.0,2.9,1031.8,0.0,25.133333,81.333333,False,Sem chuva,15.953333,0
3,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38000,-62.2060,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,131.0,4.0,1915.0,0.0,27.933333,68.000000,False,Sem chuva,21.373333,0
4,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38000,-62.2060,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,171.0,3.7,2263.9,0.0,28.833333,63.333333,False,Sem chuva,23.433333,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148863,1725445480,c5c4cb6e-3d56-3a7d-bf78-4b5f5c53b089,-12.82273,-60.2738,2024-09-29,Brasil,RONDÔNIA,Vilhena,Amazônia,VILHENA,...,127.0,5.6,2820.6,0.0,33.833333,34.666667,False,Sem chuva,58.933333,0
148864,1725445480,c5c4cb6e-3d56-3a7d-bf78-4b5f5c53b089,-12.82273,-60.2738,2024-09-29,Brasil,RONDÔNIA,Vilhena,Amazônia,VILHENA,...,130.0,5.8,2315.5,0.0,34.400000,33.333333,False,Sem chuva,59.860000,0
148865,1725445480,c5c4cb6e-3d56-3a7d-bf78-4b5f5c53b089,-12.82273,-60.2738,2024-09-29,Brasil,RONDÔNIA,Vilhena,Amazônia,VILHENA,...,96.0,5.0,1701.1,0.0,34.800000,31.333333,False,Sem chuva,60.920000,0
148866,1725445480,c5c4cb6e-3d56-3a7d-bf78-4b5f5c53b089,-12.82273,-60.2738,2024-09-29,Brasil,RONDÔNIA,Vilhena,Amazônia,VILHENA,...,312.0,8.1,801.6,0.0,30.300000,42.666667,False,Sem chuva,56.020000,0


### **🧩 5. fumaca_media**

Aqui tive que  improvisar um pouco: usei a quantidade de focos de calor no dia por município como um proxy de fumaça

In [312]:
# Contar focos por data e município | Cria um novo dataframe para essa contagem
focos_por_dia = (
    df_base_modelo_incendios
    .groupby(['municipio_pad_x', 'data'])
    .size()
    .reset_index(name='fumaca_media')
)

# Juntar essa info ao dataframe original
df_base_modelo_incendios = df_base_modelo_incendios.merge(
    focos_por_dia,
    on=['municipio_pad_x', 'data'],
    how='left'
)


In [313]:
print(focos_por_dia.shape[0])  # Total de registros após o merge

2469


In [314]:
focos_por_dia.iloc[300:305]

,municipio_pad_x,data,fumaca_media
300,CABIXI,2020-01-27,26
301,CABIXI,2020-03-06,13
302,CABIXI,2020-03-09,13
303,CABIXI,2020-05-09,24
304,CABIXI,2020-06-11,12


In [315]:
# Remove colunas duplicadas do DataFrame
df_base_modelo_incendios = df_base_modelo_incendios.loc[:, ~df_base_modelo_incendios.columns.duplicated()]

# Exibe as colunas restantes para verificar
print(df_base_modelo_incendios.columns)

Index(['id_bdq', 'foco_id', 'lat', 'lon', 'data', 'pais', 'estado',
       'municipio', 'bioma', 'municipio_pad_x', 'estacao_ref', 'hora_(utc)',
       'temp._ins._(c)', 'temp._max._(c)', 'temp._min._(c)', 'umi._ins._(%)',
       'umi._max._(%)', 'umi._min._(%)', 'pto_orvalho_ins._(c)',
       'pto_orvalho_max._(c)', 'pto_orvalho_min._(c)', 'pressao_ins._(hpa)',
       'pressao_max._(hpa)', 'pressao_min._(hpa)', 'vel._vento_(m/s)',
       'dir._vento_(m/s)', 'raj._vento_(m/s)', 'radiacao_(kj/m²)',
       'precipitacao', 'temp_media', 'umidade_relativa', 'choveu',
       'intensidade_chuva', 'risco_fogo', 'dias_sem_chuva', 'fumaca_media'],
      dtype='object')


In [316]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva,fumaca_media
0,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,10.4,17.9,7.6,21.966667,89.666667,True,Chuva moderada,11.886667,0,14
1,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.9,354.3,0.2,22.900000,90.666667,True,Chuva leve,11.960000,0,14
2,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,2.9,1031.8,0.0,25.133333,81.333333,False,Sem chuva,15.953333,0,14
3,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,4.0,1915.0,0.0,27.933333,68.000000,False,Sem chuva,21.373333,0,14
4,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.7,2263.9,0.0,28.833333,63.333333,False,Sem chuva,23.433333,0,14


### **🧩 5. risco_fogo**

Criei um índice simples e empírico com base em temperatura alta, umidade baixa e muitos dias sem chuva
**Observação:** Ajustar pesos depois, conforme a análise exploratória.

In [317]:
df_base_modelo_incendios['risco_fogo'] = (
    df_base_modelo_incendios['temp_media'] * 0.4 +
    (100 - df_base_modelo_incendios['umidade_relativa']) * 0.3 +
    df_base_modelo_incendios['dias_sem_chuva'] * 0.3
)


In [318]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva,fumaca_media
0,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,10.4,17.9,7.6,21.966667,89.666667,True,Chuva moderada,11.886667,0,14
1,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.9,354.3,0.2,22.900000,90.666667,True,Chuva leve,11.960000,0,14
2,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,2.9,1031.8,0.0,25.133333,81.333333,False,Sem chuva,15.653333,0,14
3,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,4.0,1915.0,0.0,27.933333,68.000000,False,Sem chuva,20.773333,0,14
4,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.7,2263.9,0.0,28.833333,63.333333,False,Sem chuva,22.533333,0,14


In [319]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva,fumaca_media
0,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,10.4,17.9,7.6,21.966667,89.666667,True,Chuva moderada,11.886667,0,14
1,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.9,354.3,0.2,22.900000,90.666667,True,Chuva leve,11.960000,0,14
2,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,2.9,1031.8,0.0,25.133333,81.333333,False,Sem chuva,15.653333,0,14
3,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,4.0,1915.0,0.0,27.933333,68.000000,False,Sem chuva,20.773333,0,14
4,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.7,2263.9,0.0,28.833333,63.333333,False,Sem chuva,22.533333,0,14


In [320]:
df_base_modelo_incendios.rename(columns={'chuva_categoria': 'intensidade_chuva'}, inplace=True)
# Verificar as colunas para confirmar a alteração
print(df_base_modelo_incendios.columns)

Index(['id_bdq', 'foco_id', 'lat', 'lon', 'data', 'pais', 'estado',
       'municipio', 'bioma', 'municipio_pad_x', 'estacao_ref', 'hora_(utc)',
       'temp._ins._(c)', 'temp._max._(c)', 'temp._min._(c)', 'umi._ins._(%)',
       'umi._max._(%)', 'umi._min._(%)', 'pto_orvalho_ins._(c)',
       'pto_orvalho_max._(c)', 'pto_orvalho_min._(c)', 'pressao_ins._(hpa)',
       'pressao_max._(hpa)', 'pressao_min._(hpa)', 'vel._vento_(m/s)',
       'dir._vento_(m/s)', 'raj._vento_(m/s)', 'radiacao_(kj/m²)',
       'precipitacao', 'temp_media', 'umidade_relativa', 'choveu',
       'intensidade_chuva', 'risco_fogo', 'dias_sem_chuva', 'fumaca_media'],
      dtype='object')


In [321]:
df_base_modelo_incendios.head()

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva,fumaca_media
0,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,10.4,17.9,7.6,21.966667,89.666667,True,Chuva moderada,11.886667,0,14
1,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.9,354.3,0.2,22.900000,90.666667,True,Chuva leve,11.960000,0,14
2,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,2.9,1031.8,0.0,25.133333,81.333333,False,Sem chuva,15.653333,0,14
3,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,4.0,1915.0,0.0,27.933333,68.000000,False,Sem chuva,20.773333,0,14
4,1413123146,4d3caaa1-a1e7-39d5-b179-89e9d6dbbe59,-12.38,-62.206,2020-01-16,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,3.7,2263.9,0.0,28.833333,63.333333,False,Sem chuva,22.533333,0,14


In [324]:
df_base_modelo_incendios.iloc[30520:38530]

,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva,fumaca_media
30520,1727193318,5c9e8fcf-d918-367d-be94-fefe6e3d5370,-8.57853,-63.09747,2024-10-07,Brasil,RONDÔNIA,Candeias Do Jamari,Amazônia,CANDEIAS DO JAMARI,...,5.2,3374.2,0.0,31.500000,60.000000,False,Sem chuva,31.800000,24,60
30521,1727193318,5c9e8fcf-d918-367d-be94-fefe6e3d5370,-8.57853,-63.09747,2024-10-07,Brasil,RONDÔNIA,Candeias Do Jamari,Amazônia,CANDEIAS DO JAMARI,...,5.7,3686.5,0.0,33.700000,52.000000,False,Sem chuva,35.080000,24,60
30522,1727193318,5c9e8fcf-d918-367d-be94-fefe6e3d5370,-8.57853,-63.09747,2024-10-07,Brasil,RONDÔNIA,Candeias Do Jamari,Amazônia,CANDEIAS DO JAMARI,...,6.3,3808.4,0.0,35.200000,44.333333,False,Sem chuva,37.980000,24,60
30523,1727193318,5c9e8fcf-d918-367d-be94-fefe6e3d5370,-8.57853,-63.09747,2024-10-07,Brasil,RONDÔNIA,Candeias Do Jamari,Amazônia,CANDEIAS DO JAMARI,...,6.8,2956.6,0.0,35.700000,42.000000,False,Sem chuva,38.880000,24,60
30524,1727193318,5c9e8fcf-d918-367d-be94-fefe6e3d5370,-8.57853,-63.09747,2024-10-07,Brasil,RONDÔNIA,Candeias Do Jamari,Amazônia,CANDEIAS DO JAMARI,...,4.8,2116.5,0.0,35.933333,39.333333,False,Sem chuva,39.773333,24,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38525,1652982859,4ba6c69d-53cc-37d2-8e49-025cf66fe4da,-12.42467,-63.88049,2023-09-25,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,2.8,281.9,0.0,31.533333,63.333333,False,Sem chuva,23.613333,0,21
38526,1652982862,dfae9751-e94e-33a9-9b0e-d2b742e8c57b,-12.12733,-64.17190,2023-09-25,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,0.8,138.0,0.0,25.000000,90.666667,False,Sem chuva,12.800000,0,21
38527,1652982862,dfae9751-e94e-33a9-9b0e-d2b742e8c57b,-12.12733,-64.17190,2023-09-25,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,3.1,673.3,0.0,28.233333,72.666667,False,Sem chuva,19.493333,0,21
38528,1652982862,dfae9751-e94e-33a9-9b0e-d2b742e8c57b,-12.12733,-64.17190,2023-09-25,Brasil,RONDÔNIA,Costa Marques,Amazônia,COSTA MARQUES,...,5.8,1272.2,0.0,35.233333,42.333333,False,Sem chuva,31.393333,0,21


In [325]:
df_base_modelo_incendios[df_base_modelo_incendios['dias_sem_chuva'] > 10]


,id_bdq,foco_id,lat,lon,data,pais,estado,municipio,bioma,municipio_pad_x,...,raj._vento_(m/s),radiacao_(kj/m²),precipitacao,temp_media,umidade_relativa,choveu,intensidade_chuva,risco_fogo,dias_sem_chuva,fumaca_media
267,1508086831,900bbccf-e305-399e-bfe2-e560131caed4,-11.99500,-62.46300,2020-08-18,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,2.8,194.0,0.0,20.933333,77.333333,False,Sem chuva,18.473333,11,48
268,1508086831,900bbccf-e305-399e-bfe2-e560131caed4,-11.99500,-62.46300,2020-08-18,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,1.8,842.1,0.0,22.800000,72.333333,False,Sem chuva,20.720000,11,48
269,1508086831,900bbccf-e305-399e-bfe2-e560131caed4,-11.99500,-62.46300,2020-08-18,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,2.0,1582.7,0.0,27.100000,57.333333,False,Sem chuva,26.940000,11,48
270,1508086831,900bbccf-e305-399e-bfe2-e560131caed4,-11.99500,-62.46300,2020-08-18,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,2.4,2278.6,0.0,32.033333,40.666667,False,Sem chuva,33.913333,11,48
271,1508086831,900bbccf-e305-399e-bfe2-e560131caed4,-11.99500,-62.46300,2020-08-18,Brasil,RONDÔNIA,Alta Floresta D'Oeste,Amazônia,ALTA FLORESTA DOESTE,...,7.8,2738.4,0.0,36.466667,22.666667,False,Sem chuva,41.086667,11,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148201,1722658395,f7870d3b-8705-358e-ac63-b96bfd66f4c1,-11.47477,-60.41079,2024-09-17,Brasil,RONDÔNIA,Vilhena,Amazônia,VILHENA,...,5.9,2358.3,0.0,34.533333,27.333333,False,Sem chuva,50.313333,49,585
148202,1722658395,f7870d3b-8705-358e-ac63-b96bfd66f4c1,-11.47477,-60.41079,2024-09-17,Brasil,RONDÔNIA,Vilhena,Amazônia,VILHENA,...,5.0,1788.4,0.0,35.100000,25.333333,False,Sem chuva,51.140000,49,585
148203,1722658395,f7870d3b-8705-358e-ac63-b96bfd66f4c1,-11.47477,-60.41079,2024-09-17,Brasil,RONDÔNIA,Vilhena,Amazônia,VILHENA,...,5.5,1127.4,0.0,35.066667,25.000000,False,Sem chuva,51.226667,49,585
148204,1722658395,f7870d3b-8705-358e-ac63-b96bfd66f4c1,-11.47477,-60.41079,2024-09-17,Brasil,RONDÔNIA,Vilhena,Amazônia,VILHENA,...,3.8,482.2,0.0,34.366667,26.333333,False,Sem chuva,50.546667,49,585


### **📦 Criando o DataFrame Final**

_____

In [327]:
# Lista das colunas finais desejadas
colunas_finais = [
    'data',
    'municipio',
    'estacao_ref',
    'lat',
    'lon',
    'risco_fogo',
    'temp_media',
    'umidade_relativa',
    'precipitacao',
    'dias_sem_chuva',
    'intensidade_chuva',
    'fumaca_media'
]

# Criando o DataFrame final
df_modelo_final = df_base_modelo_incendios[colunas_finais].copy()

# Conferindo as primeiras linhas
df_modelo_final.head()


,data,municipio,estacao_ref,lat,lon,risco_fogo,temp_media,umidade_relativa,precipitacao,dias_sem_chuva,intensidade_chuva,fumaca_media
0,2020-01-16,Alta Floresta D'Oeste,Cacoal,-12.38,-62.206,11.886667,21.966667,89.666667,7.6,0,Chuva moderada,14
1,2020-01-16,Alta Floresta D'Oeste,Cacoal,-12.38,-62.206,11.960000,22.900000,90.666667,0.2,0,Chuva leve,14
2,2020-01-16,Alta Floresta D'Oeste,Cacoal,-12.38,-62.206,15.653333,25.133333,81.333333,0.0,0,Sem chuva,14
3,2020-01-16,Alta Floresta D'Oeste,Cacoal,-12.38,-62.206,20.773333,27.933333,68.000000,0.0,0,Sem chuva,14
4,2020-01-16,Alta Floresta D'Oeste,Cacoal,-12.38,-62.206,22.533333,28.833333,63.333333,0.0,0,Sem chuva,14


In [329]:
df_modelo_final.iloc[30520:38630]

,data,municipio,estacao_ref,lat,lon,risco_fogo,temp_media,umidade_relativa,precipitacao,dias_sem_chuva,intensidade_chuva,fumaca_media
30520,2024-10-07,Candeias Do Jamari,Porto Velho,-8.57853,-63.09747,31.800000,31.500000,60.000000,0.0,24,Sem chuva,60
30521,2024-10-07,Candeias Do Jamari,Porto Velho,-8.57853,-63.09747,35.080000,33.700000,52.000000,0.0,24,Sem chuva,60
30522,2024-10-07,Candeias Do Jamari,Porto Velho,-8.57853,-63.09747,37.980000,35.200000,44.333333,0.0,24,Sem chuva,60
30523,2024-10-07,Candeias Do Jamari,Porto Velho,-8.57853,-63.09747,38.880000,35.700000,42.000000,0.0,24,Sem chuva,60
30524,2024-10-07,Candeias Do Jamari,Porto Velho,-8.57853,-63.09747,39.773333,35.933333,39.333333,0.0,24,Sem chuva,60
...,...,...,...,...,...,...,...,...,...,...,...,...
38625,2023-09-27,Costa Marques,Porto Velho,-12.42302,-63.87969,18.686667,28.966667,76.333333,0.0,0,Sem chuva,30
38626,2023-09-27,Costa Marques,Porto Velho,-12.42302,-63.87969,33.033333,36.333333,38.333333,0.0,0,Sem chuva,30
38627,2023-09-27,Costa Marques,Porto Velho,-12.42302,-63.87969,24.453333,29.133333,57.333333,9.6,0,Chuva moderada,30
38628,2023-09-27,Costa Marques,Porto Velho,-12.42302,-63.87969,29.673333,34.933333,47.666667,0.0,0,Sem chuva,30


In [331]:
# Verifica quantos NaNs existem por coluna
df_modelo_final.isna().sum()


data                 0
municipio            0
estacao_ref          0
lat                  0
lon                  0
risco_fogo           0
temp_media           0
umidade_relativa     0
precipitacao         0
dias_sem_chuva       0
intensidade_chuva    0
fumaca_media         0
dtype: int64

In [332]:
# Mostra todas as linhas com pelo menos um NaN
df_modelo_final[df_modelo_final.isna().any(axis=1)]


,data,municipio,estacao_ref,lat,lon,risco_fogo,temp_media,umidade_relativa,precipitacao,dias_sem_chuva,intensidade_chuva,fumaca_media


## 💾 Salvando o DataFrame para uso posterior no EDA


In [333]:

df_modelo_final.to_csv('../data/processados/modelos/df_modelo_final_incendios.csv', index=False)
print(f"✅ Arquivo salvo!!")  


✅ Arquivo salvo!!


### Formato binário, mais rápido para o Pandas

In [334]:
df_modelo_final.to_pickle('../data/processados/modelos/final_modelo_incendios.pkl')
print(f"✅ Arquivo salvo em formato PKL!!")  


✅ Arquivo salvo em formato PKL!!
